In [13]:
import torch
import numpy as np
import load_data

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torchvision import datasets, transforms

from se3cnn import SE3Convolution, SE3Dropout
from se3cnn.blocks import GatedBlock
from se3cnn.non_linearities import ScalarActivation
from se3cnn.dropout import SE3Dropout
from se3cnn import kernel
from se3cnn.filter import low_pass_filter

from tensorflow.python.framework import dtypes

In [2]:
def rotation_1(t):
    return torch.flip(t, (3, )).transpose(4, 3)

def untransformed(n, size):
    untransf = []
    for i in range(n):
        a_i = torch.zeros(1,1,size,size,size)
        for j in range(size):
            a_i[0,0,j,j,0] = np.random.randn()
            a_i[0,0,j,0,j] = a_i[0,0,j,j,0]
            a_i[0,0,0,j,j] = a_i[0,0,j,j,0]
            
        untransf.extend([(a_i, torch.tensor([1,1,1], dtype=torch.float32)), 
                         (rotation_1(a_i), torch.tensor([-1,1,1], dtype=torch.float32)), 
                         (rotation_1(rotation_1(a_i)), torch.tensor([-1,1,-1], dtype=torch.float32)), 
                         (rotation_1(rotation_1(rotation_1(a_i))), torch.tensor([1,1,-1], dtype=torch.float32))
                        ])
    return untransf

In [140]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        #self.gated = PoolGatedBlock(repr_in, repr_out, size, activation=activation, 
                                    #pool_size=pool_size, pool_stride=pool_stride, bias=bias)
        # self.conv = nn.Conv3d(1, 2, 2, 2, 1)
        self.conv1 = SE3Convolution(repr_in_1, repr_out_2, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.drop1 = nn.Dropout(prob)
        self.lin2 = nn.Linear(n_output_1, n_output_2)
        self.drop2 = nn.Dropout(prob)
        self.lin3 = nn.Linear(n_output_2, NUM_CLASSES)

    def forward(self, x):
        # x = self.gated(x)
        x = self.pool1(self.conv1(x))
        #x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) # tf.reshape(x,[batchSize,-1])
        x = F.leaky_relu(self.lin1(x))
        # x = self.drop1(x)
        # x = F.relu(self.lin2(x))
        # x = self.drop2(x)
        return self.lin2(x) #self.lin3(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    print(train_set._index_in_epoch)
    print(train_set._num_examples)
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        loss_fn = nn.MSELoss(reduction='sum')
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        loss_fn = nn.MSELoss(reduction='sum')
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            # data, target = data.to(device), target.to(device)
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) # sum up batch loss
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [141]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2)]
repr_in_2 = [(2,0),(2,1),(2,2)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 7000
n_output_1 = 3500 
n_output_2 = 7 

batch_size = 100
prob = 0.5
NUM_CLASSES = 1

In [142]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [143]:
epochs = 1000
device = torch.device('cpu')
torch.manual_seed(1)

# matrix_size = 16
# train_set = untransformed(50, matrix_size)
'''
test_set = [(a_i, torch.tensor([1,1,1], dtype=torch.float32)), 
            (rotation_1(a_i), torch.tensor([-1,1,1], dtype=torch.float32)), 
            (rotation_1(rotation_1(a_i)), torch.tensor([-1,1,-1], dtype=torch.float32)), 
            (rotation_1(rotation_1(rotation_1(a_i))), torch.tensor([1,1,-1], dtype=torch.float32))]
'''

model_hard = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_hard.parameters(), lr=learning_rate)

per_epoch = 150
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_hard, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

0
39785
Train Epoch: 1 [1000/39785 (0%)]	Loss: 2721.561279
Train Epoch: 1 [2000/39785 (0%)]	Loss: 1074.573975
Train Epoch: 1 [3000/39785 (0%)]	Loss: 973.752808
Train Epoch: 1 [4000/39785 (0%)]	Loss: 829.442627
Train Epoch: 1 [5000/39785 (0%)]	Loss: 574.667908
Train Epoch: 1 [6000/39785 (0%)]	Loss: 739.444763
Train Epoch: 1 [7000/39785 (0%)]	Loss: 558.614746
Train Epoch: 1 [8000/39785 (0%)]	Loss: 556.047913
Train Epoch: 1 [9000/39785 (0%)]	Loss: 472.887207
Train Epoch: 1 [10000/39785 (0%)]	Loss: 657.138245
Train Epoch: 1 [11000/39785 (0%)]	Loss: 715.385254
Train Epoch: 1 [12000/39785 (0%)]	Loss: 561.910278
Train Epoch: 1 [13000/39785 (0%)]	Loss: 463.862244
Train Epoch: 1 [14000/39785 (0%)]	Loss: 558.785278
Train Epoch: 1 [15000/39785 (0%)]	Loss: 489.248169
Train Epoch: 1 [16000/39785 (0%)]	Loss: 550.416016
Train Epoch: 1 [17000/39785 (0%)]	Loss: 524.253906
Train Epoch: 1 [18000/39785 (0%)]	Loss: 577.391418
Train Epoch: 1 [19000/39785 (0%)]	Loss: 370.317200
Train Epoch: 1 [20000/39785 (1

Train Epoch: 5 [6000/39785 (0%)]	Loss: 267.314728
Train Epoch: 5 [7000/39785 (0%)]	Loss: 250.307434
Train Epoch: 5 [8000/39785 (0%)]	Loss: 281.507568
Train Epoch: 5 [9000/39785 (0%)]	Loss: 182.773804
Train Epoch: 5 [10000/39785 (0%)]	Loss: 257.191284
Train Epoch: 5 [11000/39785 (0%)]	Loss: 200.231445
Train Epoch: 5 [12000/39785 (0%)]	Loss: 229.285095
Train Epoch: 5 [13000/39785 (0%)]	Loss: 229.772919
Train Epoch: 5 [14000/39785 (0%)]	Loss: 212.128021
Train Epoch: 5 [15000/39785 (0%)]	Loss: 199.787247
Train Epoch: 5 [16000/39785 (0%)]	Loss: 271.689301
Train Epoch: 5 [17000/39785 (0%)]	Loss: 221.210098
Train Epoch: 5 [18000/39785 (0%)]	Loss: 274.165039
Train Epoch: 5 [19000/39785 (0%)]	Loss: 268.127228
Train Epoch: 5 [20000/39785 (1%)]	Loss: 301.598602
Train Epoch: 5 [21000/39785 (1%)]	Loss: 254.278824
Train Epoch: 5 [22000/39785 (1%)]	Loss: 271.156189
Train Epoch: 5 [23000/39785 (1%)]	Loss: 238.583755
Train Epoch: 5 [24000/39785 (1%)]	Loss: 270.412384
Train Epoch: 5 [25000/39785 (1%)]	L

Train Epoch: 9 [11000/39785 (0%)]	Loss: 159.807724
Train Epoch: 9 [12000/39785 (0%)]	Loss: 203.979172
Train Epoch: 9 [13000/39785 (0%)]	Loss: 256.048859
Train Epoch: 9 [14000/39785 (0%)]	Loss: 234.559647
Train Epoch: 9 [15000/39785 (0%)]	Loss: 132.976486
Train Epoch: 9 [16000/39785 (0%)]	Loss: 133.622467
Train Epoch: 9 [17000/39785 (0%)]	Loss: 250.525925
Train Epoch: 9 [18000/39785 (0%)]	Loss: 147.256393
Train Epoch: 9 [19000/39785 (0%)]	Loss: 201.704956
Train Epoch: 9 [20000/39785 (1%)]	Loss: 182.743790
Train Epoch: 9 [21000/39785 (1%)]	Loss: 169.760818
Train Epoch: 9 [22000/39785 (1%)]	Loss: 213.394302
Train Epoch: 9 [23000/39785 (1%)]	Loss: 196.216064
Train Epoch: 9 [24000/39785 (1%)]	Loss: 182.808990
Train Epoch: 9 [25000/39785 (1%)]	Loss: 180.911133
Train Epoch: 9 [26000/39785 (1%)]	Loss: 185.825043
Train Epoch: 9 [27000/39785 (1%)]	Loss: 224.164673
Train Epoch: 9 [28000/39785 (1%)]	Loss: 137.947769
Train Epoch: 9 [29000/39785 (1%)]	Loss: 258.190155
Train Epoch: 9 [30000/39785 (1%

Train Epoch: 13 [13000/39785 (0%)]	Loss: 106.710693
Train Epoch: 13 [14000/39785 (0%)]	Loss: 134.318665
Train Epoch: 13 [15000/39785 (0%)]	Loss: 139.144791
Train Epoch: 13 [16000/39785 (0%)]	Loss: 133.970505
Train Epoch: 13 [17000/39785 (0%)]	Loss: 160.414810
Train Epoch: 13 [18000/39785 (0%)]	Loss: 145.912033
Train Epoch: 13 [19000/39785 (0%)]	Loss: 104.092735
Train Epoch: 13 [20000/39785 (1%)]	Loss: 200.826721
Train Epoch: 13 [21000/39785 (1%)]	Loss: 155.518921
Train Epoch: 13 [22000/39785 (1%)]	Loss: 137.370850
Train Epoch: 13 [23000/39785 (1%)]	Loss: 159.411438
Train Epoch: 13 [24000/39785 (1%)]	Loss: 1535.654663
Train Epoch: 13 [25000/39785 (1%)]	Loss: 1012.830933
Train Epoch: 13 [26000/39785 (1%)]	Loss: 5798.372070
Train Epoch: 13 [27000/39785 (1%)]	Loss: 750.386230
Train Epoch: 13 [28000/39785 (1%)]	Loss: 436.498260
Train Epoch: 13 [29000/39785 (1%)]	Loss: 425.761597
Train Epoch: 13 [30000/39785 (1%)]	Loss: 410.461060
Train Epoch: 13 [31000/39785 (1%)]	Loss: 365.869720
Train Epo

Train Epoch: 17 [15000/39785 (0%)]	Loss: 305.975708
Train Epoch: 17 [16000/39785 (0%)]	Loss: 338.773560
Train Epoch: 17 [17000/39785 (0%)]	Loss: 365.518311
Train Epoch: 17 [18000/39785 (0%)]	Loss: 254.789993
Train Epoch: 17 [19000/39785 (0%)]	Loss: 391.823395
Train Epoch: 17 [20000/39785 (1%)]	Loss: 361.137756
Train Epoch: 17 [21000/39785 (1%)]	Loss: 238.595108
Train Epoch: 17 [22000/39785 (1%)]	Loss: 311.331635
Train Epoch: 17 [23000/39785 (1%)]	Loss: 269.847229
Train Epoch: 17 [24000/39785 (1%)]	Loss: 256.381744
Train Epoch: 17 [25000/39785 (1%)]	Loss: 238.524673
Train Epoch: 17 [26000/39785 (1%)]	Loss: 196.899612
Train Epoch: 17 [27000/39785 (1%)]	Loss: 305.801971
Train Epoch: 17 [28000/39785 (1%)]	Loss: 268.996948
Train Epoch: 17 [29000/39785 (1%)]	Loss: 13286.778320
Train Epoch: 17 [30000/39785 (1%)]	Loss: 1313.130127
Train Epoch: 17 [31000/39785 (1%)]	Loss: 1179.322876
Train Epoch: 17 [32000/39785 (1%)]	Loss: 782.432617
Train Epoch: 17 [33000/39785 (1%)]	Loss: 605.575012
Train Ep

...

Train Epoch: 43 [31000/39785 (1%)]	Loss: 55.441082
Train Epoch: 43 [32000/39785 (1%)]	Loss: 70.763359
Train Epoch: 43 [33000/39785 (1%)]	Loss: 83.316055
Train Epoch: 43 [34000/39785 (1%)]	Loss: 50.247536
Train Epoch: 43 [35000/39785 (1%)]	Loss: 51.767487
Train Epoch: 43 [36000/39785 (1%)]	Loss: 62.485100
Train Epoch: 43 [37000/39785 (1%)]	Loss: 44.350361
Train Epoch: 43 [38000/39785 (1%)]	Loss: 54.388588
Train Epoch: 43 [39000/39785 (1%)]	Loss: 68.870529
39700
39785
Train Epoch: 44 [1000/39785 (0%)]	Loss: 67.387611
Train Epoch: 44 [2000/39785 (0%)]	Loss: 54.107941
Train Epoch: 44 [3000/39785 (0%)]	Loss: 48.686325
Train Epoch: 44 [4000/39785 (0%)]	Loss: 49.457703
Train Epoch: 44 [5000/39785 (0%)]	Loss: 43.195225
Train Epoch: 44 [6000/39785 (0%)]	Loss: 59.877464
Train Epoch: 44 [7000/39785 (0%)]	Loss: 48.908813
Train Epoch: 44 [8000/39785 (0%)]	Loss: 54.660458
Train Epoch: 44 [9000/39785 (0%)]	Loss: 45.513191
Train Epoch: 44 [10000/39785 (0%)]	Loss: 60.584431
Train Epoch: 44 [11000/39785

Train Epoch: 47 [36000/39785 (1%)]	Loss: 40.898792
Train Epoch: 47 [37000/39785 (1%)]	Loss: 45.971962
Train Epoch: 47 [38000/39785 (1%)]	Loss: 34.702728
Train Epoch: 47 [39000/39785 (1%)]	Loss: 42.223938
39760
39785
Train Epoch: 48 [1000/39785 (0%)]	Loss: 43.219185
Train Epoch: 48 [2000/39785 (0%)]	Loss: 34.502762
Train Epoch: 48 [3000/39785 (0%)]	Loss: 41.055058
Train Epoch: 48 [4000/39785 (0%)]	Loss: 31.658493
Train Epoch: 48 [5000/39785 (0%)]	Loss: 39.151028
Train Epoch: 48 [6000/39785 (0%)]	Loss: 229.345078
Train Epoch: 48 [7000/39785 (0%)]	Loss: 40.306435
Train Epoch: 48 [8000/39785 (0%)]	Loss: 37.408676
Train Epoch: 48 [9000/39785 (0%)]	Loss: 33.568874
Train Epoch: 48 [10000/39785 (0%)]	Loss: 33.257336
Train Epoch: 48 [11000/39785 (0%)]	Loss: 61.300510
Train Epoch: 48 [12000/39785 (0%)]	Loss: 41.643398
Train Epoch: 48 [13000/39785 (0%)]	Loss: 45.624577
Train Epoch: 48 [14000/39785 (0%)]	Loss: 42.983139
Train Epoch: 48 [15000/39785 (0%)]	Loss: 29.735014
Train Epoch: 48 [16000/3978

Train Epoch: 52 [1000/39785 (0%)]	Loss: 38.039146
Train Epoch: 52 [2000/39785 (0%)]	Loss: 31.030167
Train Epoch: 52 [3000/39785 (0%)]	Loss: 39.226330
Train Epoch: 52 [4000/39785 (0%)]	Loss: 36.567795
Train Epoch: 52 [5000/39785 (0%)]	Loss: 33.601250
Train Epoch: 52 [6000/39785 (0%)]	Loss: 28.641104
Train Epoch: 52 [7000/39785 (0%)]	Loss: 27.337687
Train Epoch: 52 [8000/39785 (0%)]	Loss: 43.180580
Train Epoch: 52 [9000/39785 (0%)]	Loss: 45.932602
Train Epoch: 52 [10000/39785 (0%)]	Loss: 40.116055
Train Epoch: 52 [11000/39785 (0%)]	Loss: 36.361622
Train Epoch: 52 [12000/39785 (0%)]	Loss: 38.304241
Train Epoch: 52 [13000/39785 (0%)]	Loss: 29.187811
Train Epoch: 52 [14000/39785 (0%)]	Loss: 33.373589
Train Epoch: 52 [15000/39785 (0%)]	Loss: 27.518009
Train Epoch: 52 [16000/39785 (0%)]	Loss: 33.967640
Train Epoch: 52 [17000/39785 (0%)]	Loss: 41.698776
Train Epoch: 52 [18000/39785 (0%)]	Loss: 47.840401
Train Epoch: 52 [19000/39785 (0%)]	Loss: 43.202614
Train Epoch: 52 [20000/39785 (1%)]	Loss:

Train Epoch: 56 [6000/39785 (0%)]	Loss: 29.782087
Train Epoch: 56 [7000/39785 (0%)]	Loss: 24.443213
Train Epoch: 56 [8000/39785 (0%)]	Loss: 22.717445
Train Epoch: 56 [9000/39785 (0%)]	Loss: 25.146513
Train Epoch: 56 [10000/39785 (0%)]	Loss: 23.430019
Train Epoch: 56 [11000/39785 (0%)]	Loss: 25.108603
Train Epoch: 56 [12000/39785 (0%)]	Loss: 348.366119
Train Epoch: 56 [13000/39785 (0%)]	Loss: 25.848816
Train Epoch: 56 [14000/39785 (0%)]	Loss: 18.311773
Train Epoch: 56 [15000/39785 (0%)]	Loss: 24.141392
Train Epoch: 56 [16000/39785 (0%)]	Loss: 27.882996
Train Epoch: 56 [17000/39785 (0%)]	Loss: 25.426882
Train Epoch: 56 [18000/39785 (0%)]	Loss: 21.784637
Train Epoch: 56 [19000/39785 (0%)]	Loss: 23.197939
Train Epoch: 56 [20000/39785 (1%)]	Loss: 18.033220
Train Epoch: 56 [21000/39785 (1%)]	Loss: 19.812561
Train Epoch: 56 [22000/39785 (1%)]	Loss: 22.534891
Train Epoch: 56 [23000/39785 (1%)]	Loss: 21.964024
Train Epoch: 56 [24000/39785 (1%)]	Loss: 18.788759
Train Epoch: 56 [25000/39785 (1%)]

KeyboardInterrupt: 

In [148]:
answer = model_hard(torch.from_numpy(train_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [149]:
answer[1]

tensor([ 0.4324,  0.8455,  0.0822,  0.1313, -0.1046, -0.4683,  0.2780],
       grad_fn=<SelectBackward>)

In [150]:
train_set._labels[1]

array([ 1.        ,  0.86292964,  0.03844497,  0.09862536, -0.08708213,
       -0.41256937,  0.25758612], dtype=float32)

In [142]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [151]:
test_name = '../deepSymmetry/data/densityMaps/density_testf'
test_set = load_data.read_data_set(test_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/densityMaps/density_testf
Extracting ../deepSymmetry/data/densityMaps/density_testf_label
(120000, 13824)


In [157]:
answer_test = model_hard(torch.from_numpy(test_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [179]:
for i in range(10):
    print(answer_test[i].data.tolist())
    print(test_set._labels[i])
    print()

[6.681089878082275, 0.8225629329681396, 0.24174951016902924, 0.033087629824876785, 0.010919880121946335, -0.17829450964927673, 0.6451407074928284]
[ 7.          0.8140826   0.16944312  0.01647427 -0.07471883 -0.16377677
  0.5252441 ]

[7.156384468078613, -0.09772703796625137, 0.8188420534133911, 0.08037760853767395, -0.6574083566665649, 0.03126903250813484, -0.26504260301589966]
[ 7.          0.01401387  0.8029993   0.18298681 -0.5421038   0.071615
 -0.15002087]

[8.394238471984863, 1.0517383813858032, 0.08680028468370438, 0.1367802619934082, 0.03753487020730972, 0.0565904825925827, -0.18338654935359955]
[ 7.0000000e+00  9.5146030e-01  4.8440255e-02  9.9439159e-05
 -3.1038227e-03  1.3755901e-02 -3.0360824e-01]

[7.138195991516113, 0.21090511977672577, 0.5391544103622437, 0.294271856546402, -0.5449842214584351, -0.480374276638031, 0.7109291553497314]
[ 7.          0.23470888  0.4653611   0.29993004 -0.5283479  -0.37522325
  0.46738502]

[8.730587005615234, 0.6358888149261475, 0.33428075

---

In [42]:
def my_dirty_loss(output, target):
    loss = torch.mean((output[0] - target[0])**2)
    return loss

def my_loss1(output, target):
    loss = torch.mean((output - target)**2)
    return loss

model = nn.Linear(2, 2)
x = torch.randn(1, 2)
target = torch.randn(1, 2)
optimizer = torch.optim.Adam(model.parameters())

for i in range(10):
    output = model(x)
    loss = my_dirty_loss(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(x, model(x))

print('---')
print(model(x))
print(model.weight.grad)

tensor([[ 0.9414, -0.5585]]) tensor([[-0.7283, -0.5375]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7258, -0.5350]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7233, -0.5325]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7208, -0.5300]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7183, -0.5275]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7158, -0.5250]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7133, -0.5225]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7108, -0.5200]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7083, -0.5175]], grad_fn=<AddmmBackward>)
tensor([[ 0.9414, -0.5585]]) tensor([[-0.7058, -0.5150]], grad_fn=<AddmmBackward>)
---
tensor([[-0.7058, -0.5150]], grad_fn=<AddmmBackward>)
tensor([[-1.0793,  0.6403],
        [-2.4632,  1.4614]])


In [197]:
def custom_loss(output, target):
    order_out = output[:, 0 : NUM_CLASSES]
    order_target = target[:, 0 : 1].type(torch.LongTensor).squeeze_()
    print(order_out.shape)
    print(order_target.shape)
    axis_out = output[:, NUM_CLASSES : NUM_CLASSES + 6]
    axis_target = target[:, 1 : 7]

    loss = nn.CrossEntropyLoss()(order_out, order_target) + nn.MSELoss(reduction='sum')(axis_out, axis_target)
    
    return loss

In [195]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SE3Convolution(repr_in_1, repr_out_2, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.drop1 = nn.Dropout(prob)
        self.lin2 = nn.Linear(n_output_1, n_output_2)
        self.drop2 = nn.Dropout(prob)
        self.lin3 = nn.Linear(n_output_2, NUM_CLASSES)

    def forward(self, x):
        # x = self.gated(x)
        x = self.pool1(self.conv1(x))
        #x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) # tf.reshape(x,[batchSize,-1])
        x = F.leaky_relu(self.lin1(x))
        # x = self.drop1(x)
        # x = F.relu(self.lin2(x))
        # x = self.drop2(x)
        return self.lin2(x) #self.lin3(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) 
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [188]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2)]
repr_in_2 = [(2,0),(2,1),(2,2)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 7000
n_output_1 = 3500 
n_output_2 = 7 

batch_size = 100
prob = 0.5
NUM_CLASSES = 1

In [189]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [198]:
epochs = 100
device = torch.device('cpu')
torch.manual_seed(1)

model_custom_loss = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_custom_loss.parameters(), lr=learning_rate)

per_epoch = 50
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_custom_loss, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

torch.Size([100, 1])
torch.Size([100])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /Users/soumith/b101_2/2019_02_08/wheel_build_dirs/wheel_3.7/pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:93